In [ ]:
%matplotlib notebook

# Dependencies

import os
import cv2
import merging_functions as mf
import numpy as np
import tracker as tr
import datetime

from matplotlib import pyplot as plt

In [ ]:
# Go through each merged file and collect the resized image in a numpy array
def get_image_sequence(video_folder, exp_name, acq_name, t_str, zstr_ar, img_dims):
    # Initialize the img numpy array
    img_seq = np.zeros((img_dims[0], img_dims[1], len(zstr_ar)), np.uint8)
    # Go over each z-stack image
    video_path = os.path.join(video_folder, f"{exp_name}_{acq_name}", f"{exp_name}_{acq_name}_{t_str}.avi")
    
    # Video capture object
    vidcap = cv2.VideoCapture(video_path)
    for zix in range(len(zstr_ar)):
        z_str = zstr_ar[zix]
        # Collect the path of the merged files
        success, frame = vidcap.read()
        # Convert to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Add to image sequence array
        img_seq[:,:,zix] = frame_gray
        
    return img_seq

In [ ]:
def collect_radial_data(roi_img, Rar_pxl):
    height, width = roi_img.shape
    centerx = np.int32(width/2)
    centery = np.int32(height/2)
    
    central_sum = np.zeros_like(Rar_pxl, dtype=np.double)
    for Rix in range(len(Rar_pxl)):
        radius = Rar_pxl[Rix]
        circle_img = np.zeros((height,width), np.uint8)
        cv2.circle(circle_img,(centerx,centery), radius,1,thickness=-1)
        masked_data = cv2.bitwise_and(roi_img, roi_img, mask=circle_img)

        central_sum[Rix] = masked_data.sum()/circle_img.sum()/255.0
        
    return central_sum

## Test for height and radius for several acq_name

In [ ]:
exp_name = "EQ59_Gly_03232021"
acq_name = "TileScan_Tp1-9_xyzt"
inner = True
img_dims = (512,512)

exp_folder = os.path.join("D:", "Tolga", "Colony Images", exp_name)
metadata = mf.collect_metadata(exp_folder, exp_name, acq_name, inner)

t_str = "t8"

zsz = metadata["dimensions"]["Z"]["NumberOfElements"]
# Find the number of digits
znum_digit = len(str(zsz))
# zstr for file path
zstr_holder = f"z%0{znum_digit}d"

# calculate z_strar for each z_str value
z_strar = np.zeros(zsz, dtype="<U6")
for zix in range(zsz):
    z_strar[zix] = zstr_holder % zix


video_folder = os.path.join(exp_folder, "Videos_ROI_Unmarked")

img_seq = get_image_sequence(video_folder, exp_name, acq_name, t_str, z_strar, img_dims)

In [ ]:
metadata

## Calculate time and radius at the surface

In [ ]:
exp_name = "EQ59_Gly_03232021"
acq_name_ar = ["TileScan_Tp1-9_xyzt",
               "TileScan_Tp10-33_xyzt"]#,
               #"TileScan_Tp34-57_xyzt"]  # Does not have start_time identifier. Metadata file is corrupt
inner = True
img_dims = (512,512)
plate_time_obj = datetime.datetime(2021, 3, 20, 23, 30)

zsurf_ixar = []
radius = []
time = []
dr = 1
Rar_pxl = np.arange(1,img_dims[0]/2, dr, dtype=np.int32)

for acq_name in acq_name_ar:
    print(f"Calculating for {acq_name}")
    exp_folder = os.path.join("D:", "Tolga", "Colony Images", exp_name)
    metadata = mf.collect_metadata(exp_folder, exp_name, acq_name, inner)
    video_folder = os.path.join(exp_folder, "Videos_ROI_Unmarked")
    
    zsz = metadata["dimensions"]["Z"]["NumberOfElements"]
    # Find the number of digits
    znum_digit = len(str(zsz))
    # zstr for file path
    zstr_holder = f"z%0{znum_digit}d"
    
    tsz = metadata["dimensions"]["T"]["NumberOfElements"]
    # Find the number of digits
    tnum_digit = len(str(tsz))
    # zstr for file path
    tstr_holder = f"t%0{tnum_digit}d"

    # calculate z_strar for each z_str value
    z_strar = np.zeros(zsz, dtype="<U6")
    for zix in range(zsz):
        z_strar[zix] = zstr_holder % zix
    
    # Choose surface z values and surface image
#     zsurf_ixar = np.zeros(tsz, dtype=np.int)
#     surf_imagear = np.zeros((img_dims[0], img_dims[1], tsz))
    # Calculate initial time
    start_time = metadata["start_time"]
    initial_time_h = (start_time - plate_time_obj).total_seconds() / 3600
    
    # Calculate dt
    timedelta_str = metadata["dimensions"]["T"]["Length"]
    hour = int(timedelta_str.split("h")[0])
    minute = int(timedelta_str.split("h")[1].split("m")[0])
    second = float(timedelta_str.split("h")[1].split("m")[1].split("s")[0])
    dt = (datetime.timedelta(hours = hour, minutes = minute, seconds=second)/tsz).total_seconds()/3600
    for tix in range(tsz):
        t_str = tstr_holder % tix
        print(f"At t={tix}/{tsz} (tstr={t_str})", end="\r", flush=True)

        img_seq = get_image_sequence(video_folder, exp_name, acq_name, t_str, z_strar, img_dims)
        total_pxl = np.zeros(zsz)
        for zix in range(zsz):
            total_pxl[zix] = img_seq[:,:,zix].sum()
        
        zsurf_ix = np.argmax(total_pxl)
        surf_image = img_seq[:,:,zsurf_ix]
        
        zsurf_ixar.append(zsurf_ix)
        central_avg = collect_radial_data(surf_image, Rar_pxl)
        ###################
        # Calculate the colony radius for each timepoint
        ###################
        # Calculate the difference of central avg
        diff_central_avg = np.diff(central_avg, prepend=0)
        # Find the last point that is positive, and call it the radius
        filterar = diff_central_avg > 0
        brightness = central_avg[filterar][-1]
        if brightness > 0.1:
            radius.append(Rar_pxl[filterar][-1])
        else:
            radius.append(0)
        
        # Add time information
        time.append(initial_time_h + dt*tix)

In [ ]:
fig,ax = plt.subplots()
ax.plot(time, radius, 'o')
ax.set_xlim(0,50)
ax.set_ylim(0,150)

## Save data to a json file

In [ ]:
import json

# TODO
#- Convert datetime to a readable datetime object for initial plate preparation
#- Convert all the data to dictionary
#- Save to a json file

## Debugging for the current timepoint. For each z sections

In [ ]:
height,width = img_seq[0].shape
dr = 1
Rar_pxl = np.arange(5,width/2, dr, dtype=np.int32)
plt_seq = np.zeros((2,len(Rar_pxl), zsz))

for zix in range(zsz):
    roi_image = img_seq[:,:,zix]
    blurred_image = cv2.GaussianBlur(roi_image, (15,15), 0).astype(np.uint8)
#     blurred_image = roi_image
    if zix%10 == 0:
        print(f"Calculating zix = {zix}/{zsz}", end="\r", flush=True)
    central_avg = collect_radial_data(blurred_image, Rar_pxl)
    plt_seq[0,:,zix] = Rar_pxl
    plt_seq[1,:,zix] = central_avg
    
fig_maxint_img, ax = plt.subplots(figsize=(4,4), dpi=200)
tracker = tr.IndexTracker(ax, plt_seq, "line", "-", 70)
#     ax.set_xlim(0,40)
ax.set_ylim(0,1)
ax.set_xlabel("Radius (pxl)")
ax.set_ylabel("Average pixel brightness")
fig_maxint_img.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()